# Week 4: Noise & Edge Detection — Lab Practice

**Topics:** Aliasing, Image Noise, Image Denoising, Quality Metrics (PSNR / SSIM), Edge Detection

This notebook accompanies the Week 4 lecture slides. We will observe aliasing artifacts, generate and denoise different types of noise, measure image quality with PSNR and SSIM, and explore edge detection operators from Sobel to Canny — all hands-on.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.ndimage import correlate, gaussian_filter, median_filter, uniform_filter
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from skimage.feature import canny

%matplotlib inline

### Environment setup

This notebook works both **locally** and on **Google Colab**.
- **Local**: images are loaded from the repository’s `images/` folder.
- **Colab**: images are automatically downloaded from GitHub on first run.

In [ ]:
import os, urllib.request

# Detect Google Colab
IN_COLAB = "google.colab" in str(get_ipython()) if hasattr(__builtins__, "__IPYTHON__") else False

# Image paths
REPO_URL = "https://raw.githubusercontent.com/HyeongminLEE/image-processing-tutorial/main"
IMAGE_DIR = "images"
COLOR_NAME = "parrots_256.jpg"
GRAY_NAME = "parrots_256_gray.jpg"
BF_FIG_NAME = "bilateral_filter_fig6.png"

if IN_COLAB:
    os.makedirs(IMAGE_DIR, exist_ok=True)
    for name in [COLOR_NAME, GRAY_NAME]:
        url = f"{REPO_URL}/{IMAGE_DIR}/{name}"
        dest = os.path.join(IMAGE_DIR, name)
        if not os.path.exists(dest):
            print(f"Downloading {name} ...")
            urllib.request.urlretrieve(url, dest)
    # Bilateral filter figure (used in Final Challenge)
    bf_url = f"{REPO_URL}/week4/{BF_FIG_NAME}"
    if not os.path.exists(BF_FIG_NAME):
        print(f"Downloading {BF_FIG_NAME} ...")
        urllib.request.urlretrieve(bf_url, BF_FIG_NAME)
    IMAGE_PATH = os.path.join(IMAGE_DIR, COLOR_NAME)
    GRAY_PATH = os.path.join(IMAGE_DIR, GRAY_NAME)
    BF_FIG_PATH = BF_FIG_NAME
else:
    IMAGE_PATH = os.path.join("..", IMAGE_DIR, COLOR_NAME)
    GRAY_PATH = os.path.join("..", IMAGE_DIR, GRAY_NAME)
    BF_FIG_PATH = BF_FIG_NAME

print(f"Image path: {IMAGE_PATH}")
print(f"Running on: {'Google Colab' if IN_COLAB else 'Local'}")

### Display helpers

Utility functions used throughout this notebook. This week adds `show_denoising_comparison` for comparing denoised results with quality metric annotations.

In [ ]:
def show_images(*imgs, titles=None, scale=4):
    """Display images in a single row.

    Grayscale (2-D) arrays automatically use a gray colormap.
    *titles* is an optional list of strings, one per image.
    """
    n = len(imgs)
    fig, axes = plt.subplots(1, n, figsize=(scale * n, scale))
    if n == 1:
        axes = [axes]
    for i, (ax, img) in enumerate(zip(axes, imgs)):
        if img.ndim == 2:
            ax.imshow(img, cmap="gray", vmin=0, vmax=255)
        else:
            ax.imshow(img)
        if titles:
            ax.set_title(titles[i])
        ax.axis("off")
    plt.tight_layout()
    plt.show()


def show_denoising_comparison(original, results, titles, scale=4):
    """Display denoised results with PSNR and SSIM annotations.

    *original* is the clean reference image.
    *results*  is a list of denoised images.
    *titles*   is a list of names (one per result).
    """
    n = len(results)
    fig, axes = plt.subplots(1, n, figsize=(scale * n, scale + 0.6))
    if n == 1:
        axes = [axes]
    for ax, img, title in zip(axes, results, titles):
        ax.imshow(img, cmap="gray", vmin=0, vmax=255)
        psnr = peak_signal_noise_ratio(original, img)
        ssim = structural_similarity(original, img)
        ax.set_title(f"{title}\nPSNR={psnr:.2f} dB  SSIM={ssim:.3f}", fontsize=10)
        ax.axis("off")
    plt.tight_layout()
    plt.show()


def show_kernel(kernel, title="Kernel", ax=None):
    """Display a small kernel as an annotated heatmap."""
    standalone = ax is None
    if standalone:
        fig, ax = plt.subplots(figsize=(3, 3))
    vmax = np.max(np.abs(kernel))
    ax.imshow(kernel, cmap="coolwarm", vmin=-vmax, vmax=vmax)
    for i in range(kernel.shape[0]):
        for j in range(kernel.shape[1]):
            val = kernel[i, j]
            text = f"{val:.2f}" if val != int(val) else str(int(val))
            ax.text(j, i, text, ha="center", va="center", fontsize=10)
    ax.set_title(title)
    ax.set_xticks([])
    ax.set_yticks([])
    if standalone:
        plt.tight_layout()
        plt.show()


def show_kernels(*kernels, titles=None):
    """Display multiple kernels as annotated heatmaps in a row."""
    n = len(kernels)
    fig, axes = plt.subplots(1, n, figsize=(3 * n, 3))
    if n == 1:
        axes = [axes]
    for i, (ax, k) in enumerate(zip(axes, kernels)):
        show_kernel(k, title=titles[i] if titles else "Kernel", ax=ax)
    plt.tight_layout()
    plt.show()


def show_kernel_3d(kernel, title=""):
    """Display a kernel as a 3D surface plot."""
    k = kernel.shape[0] // 2
    ax_range = np.arange(-k, k + 1)
    xx, yy = np.meshgrid(ax_range, ax_range)
    fig = plt.figure(figsize=(6, 5))
    ax = fig.add_subplot(111, projection="3d")
    ax.plot_surface(xx, yy, kernel, cmap="viridis")
    if title:
        ax.set_title(title)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    plt.tight_layout()
    plt.show()


def show_maps(*maps, titles=None, scale=4):
    """Display 2D float arrays with auto grayscale mapping."""
    n = len(maps)
    fig, axes = plt.subplots(1, n, figsize=(scale * n, scale))
    if n == 1:
        axes = [axes]
    for i, (ax, m) in enumerate(zip(axes, maps)):
        ax.imshow(m, cmap="gray")
        if titles:
            ax.set_title(titles[i])
        ax.axis("off")
    plt.tight_layout()
    plt.show()


def rescale_for_display(img):
    """Rescale a float image to [0, 255] uint8 for display."""
    lo, hi = img.min(), img.max()
    if hi - lo == 0:
        return np.zeros_like(img, dtype=np.uint8)
    return ((img - lo) / (hi - lo) * 255).astype(np.uint8)

---
## 1. Aliasing

When we downsample an image by simply skipping pixels, high-frequency content can **fold back** into the low-frequency range, creating false patterns called **aliasing artifacts** (Moiré patterns, jaggies).

The fix: apply a **low-pass filter** (e.g., Gaussian blur) before downsampling to remove frequencies above the new Nyquist limit.

$$\sigma \approx \frac{\text{factor}}{2}$$

In [ ]:
img_color = np.array(Image.open(IMAGE_PATH))
img_gray = np.array(Image.open(GRAY_PATH))

print(f"Color \u2014 shape: {img_color.shape}, dtype: {img_color.dtype}")
print(f"Gray  \u2014 shape: {img_gray.shape}, dtype: {img_gray.dtype}")

### Zone plate: aliasing in action

A **zone plate** contains concentric circles with increasing frequency toward the edges — a perfect test for aliasing. When we naively downsample, the high-frequency rings produce false Moiré patterns.

In [ ]:
# Create a zone plate (concentric circles with increasing frequency)
size = 256
x = np.linspace(-1, 1, size)
xx, yy = np.meshgrid(x, x)
zone_plate = (np.cos(80 * np.pi * (xx**2 + yy**2)) * 127.5 + 127.5).astype(np.uint8)

# Naive downsample: just skip pixels
factor = 4
naive_down = zone_plate[::factor, ::factor]

show_images(zone_plate, naive_down,
            titles=[f"Zone plate ({size}\u00d7{size})",
                    f"Naive \u00d7{factor} downsample ({naive_down.shape[0]}\u00d7{naive_down.shape[1]})"])

### Anti-aliased downsampling

Apply a Gaussian blur **before** downsampling to remove high frequencies that would otherwise fold back.

In [ ]:
factor = 4
sigma = factor / 2.0

# Step 1: Low-pass filter
blurred = gaussian_filter(img_gray.astype(np.float64), sigma=sigma)
blurred = np.clip(blurred, 0, 255).astype(np.uint8)

# Step 2: Downsample
naive_down = img_gray[::factor, ::factor]
aa_down = blurred[::factor, ::factor]

show_images(img_gray, naive_down, aa_down,
            titles=["Original",
                    f"Naive \u00d7{factor}",
                    f"Anti-aliased \u00d7{factor} (\u03c3={sigma})"])

---
## 2. Image Noise

Real images are always contaminated by noise. The basic noise model:

$$g(x,y) = f(x,y) + n(x,y)$$

where $f$ is the clean image and $n$ is the noise. Two common types:
- **Gaussian noise**: $n(x,y) \sim \mathcal N(0, \sigma^2)$ — additive, affects every pixel a little bit
- **Salt & Pepper noise**: random pixels flipped to 0 (pepper) or 255 (salt) — sparse but extreme

### Gaussian noise

Each pixel gets a small random perturbation drawn from a normal distribution. The parameter $\sigma$ controls the noise intensity.

In [ ]:
def add_gaussian_noise(img, sigma):
    """Add Gaussian noise with standard deviation *sigma* to an image."""
    noise = np.random.normal(0, sigma, img.shape)
    return np.clip(img.astype(np.float64) + noise, 0, 255).astype(np.uint8)

np.random.seed(42)
sigmas = [10, 25, 50]
gauss_noisy = [add_gaussian_noise(img_gray, s) for s in sigmas]

show_images(img_gray, *gauss_noisy,
            titles=["Original"] + [f"\u03c3 = {s}" for s in sigmas])

### Salt & Pepper noise

Random pixels are set to the extreme values 0 or 255. The parameter controls the **density** (fraction of affected pixels).

In [ ]:
def add_salt_pepper_noise(img, density):
    """Add salt & pepper noise with the given *density* (fraction of affected pixels)."""
    noisy = img.copy()
    mask = np.random.random(img.shape)
    noisy[mask < density / 2] = 0        # pepper
    noisy[mask > 1 - density / 2] = 255  # salt
    return noisy

np.random.seed(42)
densities = [0.05, 0.10, 0.20]
sp_noisy = [add_salt_pepper_noise(img_gray, d) for d in densities]

show_images(img_gray, *sp_noisy,
            titles=["Original"] + [f"Density = {d:.0%}" for d in densities])

### Side-by-side comparison

Gaussian noise looks like an overall **grainy texture**, while Salt & Pepper appears as scattered **black and white speckles**.

In [ ]:
np.random.seed(42)
noisy_gauss = add_gaussian_noise(img_gray, 25)
noisy_sp = add_salt_pepper_noise(img_gray, 0.10)

show_images(img_gray, noisy_gauss, noisy_sp,
            titles=["Original", "Gaussian (\u03c3=25)", "Salt & Pepper (10%)"])

---
## 3. Image Denoising

Different noise types call for different filters:
- **Gaussian noise** → Mean or Gaussian filter (averaging reduces variance)
- **Salt & Pepper noise** → Median filter (outliers can never become the median)

### Mean and Gaussian filter for Gaussian noise

Averaging $N$ independent noisy samples reduces variance by $N$:

$$\text{Var}(\bar{n}) = \frac{\sigma^2}{N}$$

A $5 \times 5$ mean filter averages 25 pixels, reducing noise $\sigma$ by a factor of $\sqrt{25} = 5$.

A Gaussian filter gives **higher weight to closer neighbors**, producing a smoother result than the box filter.

In [ ]:
np.random.seed(42)
noisy_gauss = add_gaussian_noise(img_gray, 25)

denoised_mean = uniform_filter(noisy_gauss.astype(np.float64), size=5)
denoised_mean = np.clip(denoised_mean, 0, 255).astype(np.uint8)

denoised_gauss = gaussian_filter(noisy_gauss.astype(np.float64), sigma=1.5)
denoised_gauss = np.clip(denoised_gauss, 0, 255).astype(np.uint8)

show_images(img_gray, noisy_gauss, denoised_mean, denoised_gauss,
            titles=["Original", "Noisy (\u03c3=25)", "Mean 5\u00d75", "Gaussian (\u03c3=1.5)"])

### Median filter for Salt & Pepper noise

The median filter sorts the neighborhood values and picks the **middle one**. Salt (255) or pepper (0) pixels are extreme outliers and can never become the median in a mostly-clean neighborhood.

The median filter is **non-linear** — it is NOT a convolution.

In [ ]:
np.random.seed(42)
noisy_sp = add_salt_pepper_noise(img_gray, 0.10)

denoised_median = median_filter(noisy_sp, size=5)

show_images(img_gray, noisy_sp, denoised_median,
            titles=["Original", "S&P (10%)", "Median 5\u00d75"])

### Wrong tool: mean filter on Salt & Pepper

What happens when we apply a **mean filter** to Salt & Pepper noise? Each salt pixel (255) gets **averaged** into its neighbors, spreading the extreme value into a gray blob instead of removing it.

In [ ]:
np.random.seed(42)
noisy_sp = add_salt_pepper_noise(img_gray, 0.10)

mean_on_sp = uniform_filter(noisy_sp.astype(np.float64), size=5)
mean_on_sp = np.clip(mean_on_sp, 0, 255).astype(np.uint8)

median_on_sp = median_filter(noisy_sp, size=5)

show_images(noisy_sp, mean_on_sp, median_on_sp,
            titles=["S&P (10%)", "Mean 5\u00d75 (smears!)", "Median 5\u00d75 (clean)"])

### Visual comparison: all combinations

Let's see how each filter performs on each noise type. This 2\u00d73 grid shows the strengths and weaknesses of each approach.

In [ ]:
np.random.seed(42)
noisy_gauss = add_gaussian_noise(img_gray, 25)
noisy_sp = add_salt_pepper_noise(img_gray, 0.10)

# Apply all three filters to both noise types
results = {}
for name, noisy in [("Gaussian noise", noisy_gauss), ("S&P noise", noisy_sp)]:
    mean_r = np.clip(uniform_filter(noisy.astype(np.float64), size=5), 0, 255).astype(np.uint8)
    gauss_r = np.clip(gaussian_filter(noisy.astype(np.float64), sigma=1.5), 0, 255).astype(np.uint8)
    median_r = median_filter(noisy, size=5)
    results[name] = {"Noisy": noisy, "Mean 5\u00d75": mean_r, "Gaussian \u03c3=1.5": gauss_r, "Median 5\u00d75": median_r}

fig, axes = plt.subplots(2, 4, figsize=(16, 8.5))
for row, (noise_type, imgs) in enumerate(results.items()):
    for col, (label, img) in enumerate(imgs.items()):
        axes[row, col].imshow(img, cmap="gray", vmin=0, vmax=255)
        axes[row, col].set_title(label, fontsize=11)
        axes[row, col].axis("off")
    axes[row, 0].set_ylabel(noise_type, fontsize=12, rotation=90, labelpad=10)
plt.tight_layout()
plt.show()

---
## 4. Quality Metrics

How do we **quantify** how good a denoised image is? We need metrics that compare the result to the clean original.

Three common metrics:
- **MSE** (Mean Squared Error) — average squared pixel difference
- **PSNR** (Peak Signal-to-Noise Ratio) — MSE on a log (dB) scale
- **SSIM** (Structural Similarity) — perceptual quality based on local statistics

### MSE and PSNR

$$\text{MSE} = \frac{1}{HW}\sum_{x,y}\big[f(x,y) - \hat{f}(x,y)\big]^2$$

$$\text{PSNR} = 10 \cdot \log_{10}\!\left(\frac{255^2}{\text{MSE}}\right) \;\text{dB}$$

PSNR is just MSE on a logarithmic scale. **Higher is better.** Rule of thumb: >30 dB is good quality.

In [ ]:
np.random.seed(42)
noisy_gauss = add_gaussian_noise(img_gray, 25)

# Compute from scratch
diff = img_gray.astype(np.float64) - noisy_gauss.astype(np.float64)
mse = np.mean(diff ** 2)
psnr_manual = 10 * np.log10(255**2 / mse)

# Using skimage
psnr_skimage = peak_signal_noise_ratio(img_gray, noisy_gauss)

print(f"MSE  = {mse:.2f}")
print(f"PSNR (manual)  = {psnr_manual:.2f} dB")
print(f"PSNR (skimage) = {psnr_skimage:.2f} dB")

### SSIM

SSIM compares images using **local statistics** (mean, variance, covariance) within a sliding window. It captures luminance, contrast, and structural similarity separately.

$$\text{SSIM}(x,y) = \frac{(2\mu_x\mu_y + C_1)(2\sigma_{xy} + C_2)}{(\mu_x^2 + \mu_y^2 + C_1)(\sigma_x^2 + \sigma_y^2 + C_2)}$$

- Range: $[0, 1]$, where 1 = identical
- Closer to human perception than MSE/PSNR

In [ ]:
ssim_value = structural_similarity(img_gray, noisy_gauss)
print(f"SSIM (global) = {ssim_value:.4f}")

# SSIM map: local SSIM at each pixel
_, ssim_map = structural_similarity(img_gray, noisy_gauss, full=True)

fig, axes = plt.subplots(1, 3, figsize=(14, 4))
axes[0].imshow(img_gray, cmap="gray", vmin=0, vmax=255)
axes[0].set_title("Original")
axes[1].imshow(noisy_gauss, cmap="gray", vmin=0, vmax=255)
axes[1].set_title(f"Noisy (PSNR={psnr_skimage:.1f} dB)")
im = axes[2].imshow(ssim_map, cmap="hot", vmin=0, vmax=1)
axes[2].set_title(f"SSIM map (mean={ssim_value:.3f})")
plt.colorbar(im, ax=axes[2], fraction=0.046)
for ax in axes:
    ax.axis("off")
plt.tight_layout()
plt.show()

### Evaluating denoising with metrics

Now let's **quantify** the visual comparison from Section 3 using PSNR and SSIM.

In [ ]:
np.random.seed(42)
noisy_gauss = add_gaussian_noise(img_gray, 25)
noisy_sp = add_salt_pepper_noise(img_gray, 0.10)

# Denoise Gaussian noise
gauss_denoised = np.clip(
    gaussian_filter(noisy_gauss.astype(np.float64), sigma=1.0), 0, 255
).astype(np.uint8)

# Denoise S&P noise
sp_denoised = median_filter(noisy_sp, size=3)

print("=== Gaussian noise ===")
show_denoising_comparison(
    img_gray,
    [noisy_gauss, gauss_denoised],
    ["Noisy", "Gaussian \u03c3=1.0"]
)

print("=== Salt & Pepper noise ===")
show_denoising_comparison(
    img_gray,
    [noisy_sp, sp_denoised],
    ["Noisy", "Median 3\u00d73"]
)

### Exercises

**Exercise 4.1:** Apply a **Gaussian filter** ($\sigma=1.0$) and a **Median filter** (size=3) to both `noisy_gauss` (Gaussian noise, $\sigma=25$) and `noisy_sp` (Salt & Pepper, 10%).

For each of the 4 combinations, compute the PSNR against the original image (`img_gray`). Display all four results using `show_denoising_comparison`. Which filter works best for which noise type?

*Hint:* Use `gaussian_filter` and `median_filter` from scipy, and `peak_signal_noise_ratio` from skimage. The helper `show_denoising_comparison(img_gray, [result1, result2], ["Title1", "Title2"])` will display PSNR and SSIM automatically.

In [ ]:
# YOUR CODE HERE


# Which filter works best for which noise type? Why?
# (You may write your answer in Korean.)
#

**Exercise 4.2:** Given `noisy_gauss` (Gaussian noise, $\sigma=25$), sweep the Gaussian filter's $\sigma$ across the values `[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0]`. For each $\sigma$, denoise the image and compute PSNR.

1. Plot $\sigma$ vs PSNR (use `plt.plot`)
2. Find the optimal $\sigma$ (highest PSNR)
3. Display three images: original (`img_gray`), noisy, and best-denoised

*Hint:* Use `gaussian_filter(noisy.astype(np.float64), sigma=s)`, clip to [0, 255], and cast to uint8. The optimal $\sigma$ is the one that maximizes PSNR.

In [ ]:
# YOUR CODE HERE

---
## 5. Edge Detection

An **edge** is a significant local change in intensity. We detect edges by computing **derivatives** of the image.

The **gradient** at each pixel:

$$\nabla f = \begin{bmatrix} G_x \\ G_y \end{bmatrix} \quad\quad
\text{Magnitude: } |\nabla f| = \sqrt{G_x^2 + G_y^2} \quad\quad
\text{Direction: } \theta = \arctan\!\left(\frac{G_y}{G_x}\right)$$

### 1D derivative intuition

Before working with 2D images, let's look at how derivatives detect edges in a 1D signal. The **first derivative** peaks at edges, while the **second derivative** crosses zero.

In [ ]:
# Extract a horizontal line from the image
row = 128
profile = img_gray[row, :].astype(np.float64)

# 1st derivative (central difference)
d1 = np.convolve(profile, [-1, 0, 1], mode="same")

# 2nd derivative
d2 = np.convolve(profile, [1, -2, 1], mode="same")

fig, axes = plt.subplots(3, 1, figsize=(10, 7), sharex=True)
axes[0].plot(profile)
axes[0].set_ylabel("Intensity")
axes[0].set_title(f"1D profile (row {row})")

axes[1].plot(d1)
axes[1].axhline(0, color="gray", linewidth=0.5)
axes[1].set_ylabel("1st derivative")
axes[1].set_title("First derivative (peaks at edges)")

axes[2].plot(d2)
axes[2].axhline(0, color="gray", linewidth=0.5)
axes[2].set_ylabel("2nd derivative")
axes[2].set_title("Second derivative (zero crossings at edges)")
axes[2].set_xlabel("Column")

plt.tight_layout()
plt.show()

### Sobel operator (1st derivative)

The Sobel operator computes the image gradient using two 3\u00d73 kernels. Each kernel combines **smoothing** in one direction with a **derivative** in the other:

$$G_x = \begin{bmatrix}-1 & 0 & 1\\-2 & 0 & 2\\-1 & 0 & 1\end{bmatrix} \quad\quad
G_y = \begin{bmatrix}-1 & -2 & -1\\0 & 0 & 0\\1 & 2 & 1\end{bmatrix}$$

In [ ]:
sobel_x = np.array([[-1, 0, 1],
                     [-2, 0, 2],
                     [-1, 0, 1]], dtype=np.float64)

sobel_y = np.array([[-1, -2, -1],
                     [ 0,  0,  0],
                     [ 1,  2,  1]], dtype=np.float64)

show_kernels(sobel_x, sobel_y, titles=["Sobel $G_x$", "Sobel $G_y$"])

In [ ]:
gx = correlate(img_gray.astype(np.float64), sobel_x, mode="constant", cval=0)
gy = correlate(img_gray.astype(np.float64), sobel_y, mode="constant", cval=0)
magnitude = np.sqrt(gx**2 + gy**2)

show_images(img_gray, rescale_for_display(gx), rescale_for_display(gy), rescale_for_display(magnitude),
            titles=["Original", "$G_x$ (horizontal edges)", "$G_y$ (vertical edges)", "Magnitude"])

### Laplacian (2nd derivative)

The Laplacian computes the sum of second partial derivatives using a **single kernel**. Edges appear as **zero crossings**.

$$\nabla^2 f = \frac{\partial^2 f}{\partial x^2} + \frac{\partial^2 f}{\partial y^2}$$

Two variants:
- **4-connected**: $\begin{bmatrix}0 & 1 & 0\\1 & -4 & 1\\0 & 1 & 0\end{bmatrix}$ — horizontal & vertical
- **8-connected**: $\begin{bmatrix}1 & 1 & 1\\1 & -8 & 1\\1 & 1 & 1\end{bmatrix}$ — includes diagonals (more isotropic)

In [ ]:
lap4 = np.array([[0,  1, 0],
                  [1, -4, 1],
                  [0,  1, 0]], dtype=np.float64)

lap8 = np.array([[1,  1, 1],
                  [1, -8, 1],
                  [1,  1, 1]], dtype=np.float64)

edges4 = correlate(img_gray.astype(np.float64), lap4, mode="constant", cval=0)
edges8 = correlate(img_gray.astype(np.float64), lap8, mode="constant", cval=0)

show_images(img_gray, rescale_for_display(edges4), rescale_for_display(edges8),
            titles=["Original", "Laplacian (4-connected)", "Laplacian (8-connected)"])

### Laplacian of Gaussian (LoG)

Raw derivatives amplify noise. The solution: **smooth first, then differentiate**. By linearity, we can combine these into a single kernel:

$$\nabla^2(G_\sigma * f) = (\nabla^2 G_\sigma) * f$$

The LoG kernel has a characteristic **Mexican hat** shape: a positive center surrounded by a negative ring.

In [ ]:
def make_log_kernel(size, sigma):
    """Create a Laplacian of Gaussian (LoG) kernel."""
    k = size // 2
    ax = np.arange(-k, k + 1, dtype=np.float64)
    xx, yy = np.meshgrid(ax, ax)
    r2 = xx**2 + yy**2
    # LoG formula
    kernel = -(1 / (np.pi * sigma**4)) * (1 - r2 / (2 * sigma**2)) * np.exp(-r2 / (2 * sigma**2))
    kernel -= kernel.mean()  # ensure zero-sum
    return kernel

log_kernel = make_log_kernel(15, sigma=2.0)
show_kernel_3d(log_kernel, title="LoG kernel (\u03c3=2.0) \u2014 Mexican hat")

In [ ]:
log_edges = correlate(img_gray.astype(np.float64), log_kernel, mode="constant", cval=0)

show_images(img_gray, rescale_for_display(log_edges),
            titles=["Original", "LoG edges (\u03c3=2.0)"])

### Canny edge detection

The Canny detector is a complete **pipeline** that produces clean, thin, connected edges:

1. **Gaussian blur** — smooth the image (controlled by $\sigma$)
2. **Gradient computation** — Sobel for magnitude and direction
3. **Non-maximum suppression** — keep only local maxima along the gradient direction → 1-pixel-thin edges
4. **Hysteresis thresholding** — two thresholds ($T_\text{high}$, $T_\text{low}$) connect strong and weak edges

In [ ]:
# Canny expects float image in [0, 1]
img_float = img_gray / 255.0

edges_s1 = canny(img_float, sigma=1.0).astype(np.uint8) * 255
edges_s2 = canny(img_float, sigma=2.0).astype(np.uint8) * 255
edges_s4 = canny(img_float, sigma=4.0).astype(np.uint8) * 255

show_images(img_gray, edges_s1, edges_s2, edges_s4,
            titles=["Original", "Canny \u03c3=1.0", "Canny \u03c3=2.0", "Canny \u03c3=4.0"])

### Comparison: Sobel vs Laplacian vs LoG vs Canny

In [ ]:
sobel_mag = rescale_for_display(np.sqrt(
    correlate(img_gray.astype(np.float64), sobel_x, mode="constant", cval=0)**2 +
    correlate(img_gray.astype(np.float64), sobel_y, mode="constant", cval=0)**2
))
lap_result = rescale_for_display(correlate(img_gray.astype(np.float64), lap4, mode="constant", cval=0))
log_result = rescale_for_display(correlate(img_gray.astype(np.float64), make_log_kernel(15, 2.0), mode="constant", cval=0))
canny_result = canny(img_gray / 255.0, sigma=2.0).astype(np.uint8) * 255

show_images(sobel_mag, lap_result, log_result, canny_result,
            titles=["Sobel magnitude", "Laplacian", "LoG (\u03c3=2)", "Canny (\u03c3=2)"])

### Exercises

**Exercise 5.1:** Apply the Sobel operator to the grayscale image (`img_gray`) to compute $G_x$ and $G_y$. Then compute:
- **Gradient magnitude**: $|\nabla f| = \sqrt{G_x^2 + G_y^2}$
- **Gradient direction**: $\theta = \text{arctan2}(G_y,\, G_x)$

Display three images side by side: the original, the gradient magnitude, and the gradient direction.

*Hint:* Use `sobel_x`, `sobel_y` with `scipy.ndimage.correlate`. For direction, use `np.arctan2(gy, gx)`. Display the magnitude with `rescale_for_display` and the direction with `cmap='hsv'` (since angles map naturally to a circular colormap).

In [ ]:
# YOUR CODE HERE

**Exercise 5.2:** Add Gaussian noise ($\sigma=30$) to `img_gray`. Then apply four edge detectors to the **noisy** image and display the results:

1. **Sobel** magnitude ($\sqrt{G_x^2 + G_y^2}$)
2. **Laplacian** (4-connected kernel `lap4`)
3. **LoG** (use `make_log_kernel(15, 2.0)`)
4. **Canny** (`canny(img / 255.0, sigma=2.0)`)

Which operators handle noise well, and which ones break down? Why?

*Hint:* Use `add_gaussian_noise`, the Sobel/Laplacian/LoG patterns from the demos above, and `rescale_for_display` for Sobel/Laplacian/LoG results. Canny returns a boolean array — multiply by 255 for display.

In [ ]:
# YOUR CODE HERE


# Which operators handle noise well? Why?
# (You may write your answer in Korean.)
#

---
## Final Challenge

### Bilateral Filter: Edge-Preserving Denoising

All the filters we have used so far (mean, Gaussian, median) share a common limitation: they smooth **everything**, including edges. The Gaussian filter, for example, reduces noise but also blurs sharp boundaries.

The **bilateral filter** solves this by using two Gaussian weights:
- **Spatial weight** $G_{\sigma_s}(\|p - q\|)$ — nearby pixels contribute more (same as Gaussian filter)
- **Range weight** $G_{\sigma_r}(|I_p - I_q|)$ — pixels with **similar intensity** contribute more

$$\text{BF}[I]_p = \frac{1}{W_p}\sum_{q \in \mathcal N(p)} G_{\sigma_s}(\|p-q\|)\;\cdot\;G_{\sigma_r}(|I_p - I_q|)\;\cdot\;I_q$$

where $W_p = \sum_q G_{\sigma_s} \cdot G_{\sigma_r}$ is a normalization factor.

**Key insight**: At an edge, pixels on the opposite side have very different intensity, so their range weight becomes near-zero. The filter effectively **stops at edges** while still averaging similar pixels.

The figure below (from Durand & Dorsey, 2002) illustrates the bilateral filtering process for a single pixel:

In [ ]:
bf_fig = Image.open(BF_FIG_PATH)
fig, ax = plt.subplots(figsize=(14, 3))
ax.imshow(bf_fig)
ax.axis("off")
ax.set_title("Bilateral filtering: input \u2192 spatial kernel \u2192 range kernel \u2192 combined weight \u2192 output",
             fontsize=11)
plt.tight_layout()
plt.show()

**Your task:** Complete the `bilateral_filter` function below. The outer loop, padding, and spatial weight precomputation are provided. You need to fill in the **4 lines** inside the loop:

1. Compute **range weights**: apply a Gaussian to the intensity difference between each neighbor and the center pixel
2. Combine spatial and range weights
3. Normalize so the weights sum to 1
4. Compute the weighted average

Then apply your bilateral filter to a noisy image and compare with a standard Gaussian filter using PSNR.

*Hint:* The range weight for each neighbor is $\exp\!\left(-\frac{(I_\text{neighbor} - I_\text{center})^2}{2\sigma_r^2}\right)$. You can compute this for the entire neighborhood array at once using NumPy broadcasting.

In [ ]:
def bilateral_filter(img, d=5, sigma_s=2.0, sigma_r=25.0):
    """Bilateral filter: edge-preserving denoising.

    Parameters
    ----------
    img     : 2D uint8 array (grayscale image)
    d       : kernel diameter (must be odd)
    sigma_s : spatial Gaussian standard deviation
    sigma_r : range (intensity) Gaussian standard deviation
    """
    img_f = img.astype(np.float64)
    h, w = img_f.shape
    r = d // 2
    output = np.zeros_like(img_f)

    # Pre-compute spatial Gaussian weights (same for every pixel)
    ax = np.arange(-r, r + 1, dtype=np.float64)
    xx, yy = np.meshgrid(ax, ax)
    spatial_weights = np.exp(-(xx**2 + yy**2) / (2 * sigma_s**2))

    # Pad image for border handling
    padded = np.pad(img_f, r, mode="reflect")

    for i in range(h):
        for j in range(w):
            # Extract the local neighborhood
            neighborhood = padded[i : i + d, j : j + d]
            center_val = img_f[i, j]

            # YOUR CODE HERE
            # 1. range_weights = Gaussian of (neighborhood - center_val)
            # 2. combined_weights = spatial_weights * range_weights
            # 3. Normalize combined_weights so they sum to 1
            # 4. output[i, j] = weighted sum of neighborhood

    return np.clip(output, 0, 255).astype(np.uint8)

Test your implementation and compare with a standard Gaussian filter:

In [ ]:
# YOUR CODE HERE
# 1. Generate a noisy image: add_gaussian_noise(img_gray, 25) with np.random.seed(42)
# 2. Apply your bilateral_filter (try d=5, sigma_s=2.0, sigma_r=25.0)
# 3. Apply gaussian_filter with sigma=1.0 for comparison
# 4. Use show_denoising_comparison to display: noisy, Gaussian-denoised, bilateral-denoised